# Run Objective:

###### Run0128
-  Based on Run0126f but with the dataset used switched from `unstable_training_gen3_7D_nions0_flat_filter8.h5` to `gen3_7D_nions0_flat_filter8.h5`.

In [ ]:
# DO NOT RUN THIS PART

In [ ]:
import os
import sys
import time

# Gets the current file name. Useful for procedurally generating output/log files.
file_name =  os.path.basename(sys.argv[0][:-3])
print(file_name)

if file_name == "ipykernel_launcher":
    print("This is the Jupyter version.")
    print("Now MANUALLY run the next two cells!")
    print("STOP! This should not be in your code!!")
    exit(0)
    time.sleep(10)
    print("Testing if script has really stopped.")
else:
    print("This is the Atom version")

In [1]:
# RUN ONLY IN JUPYTER!!
# Start here (manual)

In [2]:
%%javascript
IPython.notebook.kernel.execute('file_name = "' + IPython.notebook.notebook_name + '"');

<IPython.core.display.Javascript object>

In [3]:
print(file_name)

2018-10-18_Run0128.ipynb


In [4]:
file_name = file_name[:-6]
print(file_name)

is_Jupyter = True
print(is_Jupyter)

2018-10-18_Run0128
True


In [5]:
# Same code for both ATOM & JUPYTER from now (Run all cells below now)

In [6]:
'''
Late Fusion Module (test) - Functional API
'''

# Multiple Inputs
import keras
from keras.optimizers import RMSprop, adam, Adam
from keras.initializers import TruncatedNormal, glorot_normal, Constant
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers.merge import concatenate
from keras import regularizers
from keras import backend as K
from keras.backend import switch
import pandas
import numpy
import sys
import os
from copy import deepcopy
import tensorflow as tf
from keras.utils.generic_utils import get_custom_objects
from keras.layers.advanced_activations import ThresholdedReLU

keras.backend.clear_session()

Using TensorFlow backend.


In [7]:
# Define new Metric: rmse = Root Mean Square Error
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square( y_true-y_pred )))

In [8]:
# Modify for ATOM use

In [9]:
if is_Jupyter == True:
    pass
else:
    # Gets the current file name. Useful for procedurally generating output/log files.
    file_name =  os.path.basename(sys.argv[0][:-3])

In [10]:
# Define neural network parameters
batch_size = 10
#num_classes = 1
epochs = 100

In [11]:
# Load Data (which is in HDF5 or .h5 format)
store = pandas.HDFStore("gen3_7D_nions0_flat_filter8.h5")
target_df = store['/output/efeETG_GB'].to_frame()  # This one is relatively easy to train
input_df = store['input']

# Puts inputs and outputs in the same pandas dataframe.
# Also only keeps overlapping entries.
joined_dataFrame = target_df.join(input_df)

# Make a copy of joined_dataFrame for later use
joined_dataFrame_original = deepcopy(joined_dataFrame)


# *************************************************************************** #
# Normalize data by standard deviation and mean-centering the data
# Standard configuration
joined_dataFrame['efeETG_GB'] = (joined_dataFrame['efeETG_GB'] - joined_dataFrame['efeETG_GB'].mean()) / joined_dataFrame['efeETG_GB'].std()
joined_dataFrame['Ati'] = (joined_dataFrame['Ati'] - joined_dataFrame['Ati'].mean()) / joined_dataFrame['Ati'].std()
joined_dataFrame['Ate'] = (joined_dataFrame['Ate'] - joined_dataFrame['Ate'].mean()) / joined_dataFrame['Ate'].std()
joined_dataFrame['An'] = (joined_dataFrame['An'] - joined_dataFrame['An'].mean()) / joined_dataFrame['An'].std()
joined_dataFrame['q'] = (joined_dataFrame['q'] - joined_dataFrame['q'].mean()) / joined_dataFrame['q'].std()
joined_dataFrame['smag'] = (joined_dataFrame['smag'] - joined_dataFrame['smag'].mean()) / joined_dataFrame['smag'].std()
joined_dataFrame['x'] = (joined_dataFrame['x'] - joined_dataFrame['x'].mean()) / joined_dataFrame['x'].std()
joined_dataFrame['Ti_Te'] = (joined_dataFrame['Ti_Te'] - joined_dataFrame['Ti_Te'].mean()) / joined_dataFrame['Ti_Te'].std()

# Shuffles dataset
shuffled_joined_dataFrame = joined_dataFrame.reindex(numpy.random.permutation(
                                                joined_dataFrame.index))

# Creates a pandas dataframe for the outputs
shuffled_clean_output_df = shuffled_joined_dataFrame['efeETG_GB']

# Make a copy of shuffled_joined_dataFrame for later use
shuffled_joined_dataFrame_base = deepcopy(shuffled_joined_dataFrame)



# *************************************************************************** #
# Creates a pandas dataframe for the inputs (7D)
shuffled_clean_input_df_7D = shuffled_joined_dataFrame.drop('efeETG_GB', axis=1)

# Creates training dataset (90% of total data) for outputs
y_train = shuffled_clean_output_df.iloc[:int(
    numpy.round(len(shuffled_clean_output_df)*0.9))]

# Creates training dataset (90% of total data) for inputs
x_train = shuffled_clean_input_df_7D.iloc[:int(
    numpy.round(len(shuffled_clean_input_df_7D)*0.9))]

# Creates testing dataset (10% of total data) for outputs
y_test = shuffled_clean_output_df.iloc[int(
    numpy.round(len(shuffled_clean_output_df)*0.9)):]

# Creates testing dataset (10% of total data) for inputs
x_test = shuffled_clean_input_df_7D.iloc[int(
    numpy.round(len(shuffled_clean_input_df_7D)*0.9)):]
# *************************************************************************** #


# Deletes pandas dataframes that are no longer needed
del target_df, input_df

# Closes the HDFStore. This is good practice.
store.close()


In [12]:
joined_dataFrame_original.describe(include='all')

,efeETG_GB,Ati,Ate,An,q,smag,x,Ti_Te
count,9.676800e+06,9.676800e+06,9.676800e+06,9.676800e+06,9.676800e+06,9.676800e+06,9.676800e+06,9.676800e+06
mean,1.062595e+01,5.750000e+00,5.750000e+00,9.583333e-01,4.466000e+00,1.595000e+00,5.212500e-01,1.141429e+00
std,3.192191e+01,3.628590e+00,3.628590e+00,2.868350e+00,4.352977e+00,1.681733e+00,2.908366e-01,7.100791e-01
min,-6.307309e+00,1.000000e-14,1.000000e-14,-5.000000e+00,6.600000e-01,-1.000000e+00,9.000000e-02,2.500000e-01
25%,0.000000e+00,3.312500e+00,3.312500e+00,-2.500000e-01,1.500000e+00,4.000000e-01,3.000000e-01,5.000000e-01
50%,0.000000e+00,5.375000e+00,5.375000e+00,1.250000e+00,2.750000e+00,1.250000e+00,5.100000e-01,1.000000e+00
75%,0.000000e+00,7.437500e+00,7.437500e+00,2.625000e+00,5.000000e+00,2.750000e+00,7.275000e-01,1.660000e+00
max,4.052581e+02,1.400000e+01,1.400000e+01,6.000000e+00,1.500000e+01,5.000000e+00,9.900000e-01,2.500000e+00


In [13]:
shuffled_clean_input_df_7D.describe(include='all')

,Ati,Ate,An,q,smag,x,Ti_Te
count,9.676800e+06,9.676800e+06,9.676800e+06,9.676800e+06,9.676800e+06,9.676800e+06,9.676800e+06
mean,7.998305e-16,3.065860e-16,2.985031e-16,4.877932e-17,4.124273e-17,1.982541e-18,1.298564e-16
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-1.584637e+00,-1.584637e+00,-2.077269e+00,-8.743442e-01,-1.543051e+00,-1.482791e+00,-1.255393e+00
25%,-6.717485e-01,-6.717485e-01,-4.212643e-01,-6.813728e-01,-7.105766e-01,-7.607363e-01,-9.033199e-01
50%,-1.033459e-01,-1.033459e-01,1.016845e-01,-3.942130e-01,-2.051455e-01,-3.868153e-02,-1.991730e-01
75%,4.650566e-01,4.650566e-01,5.810542e-01,1.226747e-01,6.867916e-01,7.091610e-01,7.303009e-01
max,2.273610e+00,2.273610e+00,1.757689e+00,2.419953e+00,2.024697e+00,1.611730e+00,1.913268e+00


In [14]:
# Creates a pandas dataframe for the inputs
shuffled_clean_input_df_1 = shuffled_clean_input_df_7D.drop('Ate', axis=1)

In [15]:
shuffled_clean_input_df_1.describe(include='all')

,Ati,An,q,smag,x,Ti_Te
count,9.676800e+06,9.676800e+06,9.676800e+06,9.676800e+06,9.676800e+06,9.676800e+06
mean,7.998305e-16,2.985031e-16,4.877932e-17,4.124273e-17,1.982541e-18,1.298564e-16
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-1.584637e+00,-2.077269e+00,-8.743442e-01,-1.543051e+00,-1.482791e+00,-1.255393e+00
25%,-6.717485e-01,-4.212643e-01,-6.813728e-01,-7.105766e-01,-7.607363e-01,-9.033199e-01
50%,-1.033459e-01,1.016845e-01,-3.942130e-01,-2.051455e-01,-3.868153e-02,-1.991730e-01
75%,4.650566e-01,5.810542e-01,1.226747e-01,6.867916e-01,7.091610e-01,7.303009e-01
max,2.273610e+00,1.757689e+00,2.419953e+00,2.024697e+00,1.611730e+00,1.913268e+00


In [16]:
print(shuffled_clean_input_df_1.shape)

(9676800, 6)


In [17]:
shuffled_clean_input_df_2 = shuffled_clean_input_df_7D.drop('Ati', axis=1)
shuffled_clean_input_df_2 = shuffled_clean_input_df_2.drop('An', axis=1)
shuffled_clean_input_df_2 = shuffled_clean_input_df_2.drop('q', axis=1)
shuffled_clean_input_df_2 = shuffled_clean_input_df_2.drop('smag', axis=1)
shuffled_clean_input_df_2 = shuffled_clean_input_df_2.drop('x', axis=1)
shuffled_clean_input_df_2 = shuffled_clean_input_df_2.drop('Ti_Te', axis=1)

In [18]:
shuffled_clean_input_df_2.describe(include='all')

,Ate
count,9.676800e+06
mean,3.065860e-16
std,1.000000e+00
min,-1.584637e+00
25%,-6.717485e-01
50%,-1.033459e-01
75%,4.650566e-01
max,2.273610e+00


In [19]:
print(shuffled_clean_input_df_2.shape)

(9676800, 1)


In [20]:
# *************************************************************************** #
# Branch 1

# Creates training dataset (90% of total data) for inputs
x_train_1 = shuffled_clean_input_df_1.iloc[:int(
    numpy.round(len(shuffled_clean_input_df_1)*0.9))]

# Creates testing dataset (10% of total data) for inputs
x_test_1 = shuffled_clean_input_df_1.iloc[int(
    numpy.round(len(shuffled_clean_input_df_1)*0.9)):]
# *************************************************************************** #

In [21]:
# *************************************************************************** #
# Branch 2

# Creates training dataset (90% of total data) for inputs
x_train_2 = shuffled_clean_input_df_2.iloc[:int(
    numpy.round(len(shuffled_clean_input_df_2)*0.9))]

# Creates testing dataset (10% of total data) for inputs
x_test_2 = shuffled_clean_input_df_2.iloc[int(
    numpy.round(len(shuffled_clean_input_df_2)*0.9)):]
# *************************************************************************** #

In [22]:
# branch1
visible_branch1 = Input(shape=(6,))
hidden1_branch1 = Dense(30,
        activation='tanh',
        kernel_initializer='glorot_normal',
        kernel_regularizer=regularizers.l2(0.00005),
        use_bias=True, bias_initializer='glorot_normal')(visible_branch1)
hidden2_branch1 = Dense(30,
        activation='tanh',
        kernel_initializer='glorot_normal',
        kernel_regularizer=regularizers.l2(0.00005),
        use_bias=True, bias_initializer='glorot_normal')(hidden1_branch1)
theta_branch1 = Dense(1,
        activation='tanh',
        kernel_initializer='glorot_normal',
        kernel_regularizer=regularizers.l2(0.00005),
        use_bias=True, bias_initializer='glorot_normal')(hidden2_branch1)
theta_branch1_feeder = Dense(1,
        activation='linear',
        kernel_initializer=Constant(value=-1),
        bias_initializer='Zeros',
        trainable=False)(theta_branch1)

# branch2 (Ate input)
visible_branch2 = Input(shape=(1,))
visible_branch2_feeder = Dense(1,
                activation='linear')(visible_branch2)

# addition_pooling (effectively subtraction though...)
addition_pooling = keras.layers.Add()([theta_branch1_feeder, visible_branch2_feeder])


TR = Dense(1, activation='relu',
           kernel_initializer='Ones',
           bias_initializer='Zeros',
           trainable=False)(addition_pooling)

# merge input models
#merge = concatenate([hidden2_branch1, TR])

# interpretation model
output = Dense(1, activation='linear',
           kernel_initializer='Ones',
           bias_initializer='Zeros',
           trainable=False)(TR)

model = Model(inputs=[visible_branch1, visible_branch2], outputs=output)

# summarize layers
print(model.summary())

# plot graph
plot_model(model, 'ModelPlots/' + str(file_name) + '_model_plot.png')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 6)            0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 30)           210         input_1[0][0]                    
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 30)           930         dense_1[0][0]                    
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 1)            31          dense_2[0][0]                    
__________________________________________________________________________________________________
input_2 (I

OSError: `pydot` failed to call GraphViz.Please install GraphViz (https://www.graphviz.org/) and ensure that its executables are in the $PATH.

In [ ]:
model.compile(loss='mean_squared_error',   #categorical_crossentropy
              optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999),
              metrics=["mae", "mean_squared_error", rmse])

# Add CallBacks (including TensorBoard)
tbCallBack = keras.callbacks.TensorBoard(
        log_dir='TensorBoard_logs/' + str(file_name), write_graph = False, write_images=False, write_grads=False)
EarlyStoppingCallBack = keras.callbacks.EarlyStopping(
        monitor='val_rmse', min_delta=0, patience=15, verbose=0, mode='auto')

history = model.fit([x_train_1, x_train_2],
                    y = y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    shuffle=True,
                    verbose=2,
                    # validation_data=(x_test, y_test),
                    validation_data=([x_test_1, x_test_2], y_test),
                    callbacks=[tbCallBack, EarlyStoppingCallBack])

# score = model.evaluate(x_test, y_test, verbose=0)
score = model.evaluate([x_test_1, x_test_2], y_test, verbose=0)

print('Test loss:', score[0])
print('val_mean_absolute_error:', score[1])

print("score")
print(score)

print("model.metrics_names")
print(model.metrics_names)

# creates a HDF5 file 'my_model.h5'
model.save("./Saved-Networks/" + str(file_name) +".h5")

# Create output file
OutputFile = open("./Loss-Values/" +str(file_name) +".txt", "w+")
OutputFile.write("Test loss: " + str(score[0]) + "\n")
OutputFile.write("val_mean_absolute_error: " +str(score[1]) + "\n")
OutputFile.write("val_mean_squared_error: " +str(score[2]) + "\n")
OutputFile.write("RMSE: " +str(score[3]) + "\n")
OutputFile.close()

del history
del model


Train on 8709120 samples, validate on 967680 samples
Epoch 1/100
 - 569s - loss: 0.9447 - mean_absolute_error: 0.5309 - mean_squared_error: 0.9447 - rmse: 0.7869 - val_loss: 1.4982 - val_mean_absolute_error: 0.6580 - val_mean_squared_error: 1.4982 - val_rmse: 0.7296
Epoch 2/100
